In [3]:
import os
import pickle
import numpy as np

ben_dirs = ['train/Benign_Dumps','validation/Benign_Dumps','test/Benign_Dumps']
mal_dirs = ['train/Malware_Dumps','validation/Malware_Dumps','test/Malware_Dumps']
ben_files = []
mal_files = []

shuffled_dir = 'Updated'

file_dirs = {}
for dir in ben_dirs:
    files = os.listdir(dir)
    new_file_dirs = {f:dir for f in files}
    file_dirs.update(new_file_dirs)
    ben_files += files
    
for dir in mal_dirs:
    files = os.listdir(dir)
    new_file_dirs = {f:dir for f in files}
    file_dirs.update(new_file_dirs)
    mal_files += files
    
    
tr_ben = 380
tr_mal = 619
val_ben = 100
val_mal = 100
test_ben = 100
test_mal = 105

def shuffle(files):
    rng = np.random.RandomState(150)
    indices = np.arange(len(files))
    rng.shuffle(indices)
    files = [files[i] for i in indices]

    return files

shuffled_ben_files = shuffle(ben_files)
shuffled_mal_files = shuffle(mal_files)

new_ben_dirs = [shuffled_dir + '/' + dir_name for dir_name in ben_dirs]
new_mal_dirs = [shuffled_dir + '/' + dir_name for dir_name in mal_dirs]

for d in new_ben_dirs+new_mal_dirs:
    os.makedirs(d)

import shutil
index = 0
while index < tr_ben:
    shutil.copy(file_dirs[shuffled_ben_files[index]]+'/'+ shuffled_ben_files[index],new_ben_dirs[0]+'/'+shuffled_ben_files[index])
    index += 1
while index < tr_ben + val_ben:
    shutil.copy(file_dirs[shuffled_ben_files[index]]+'/'+ shuffled_ben_files[index],new_ben_dirs[1]+'/'+shuffled_ben_files[index])
    index += 1
while index < tr_ben + val_ben + test_ben:
    shutil.copy(file_dirs[shuffled_ben_files[index]]+'/'+ shuffled_ben_files[index],new_ben_dirs[2]+'/'+shuffled_ben_files[index])
    index += 1
    
index = 0
while index < tr_mal:
    shutil.copy(file_dirs[shuffled_mal_files[index]]+'/'+ shuffled_mal_files[index],new_mal_dirs[0]+'/'+shuffled_mal_files[index])
    index += 1
while index < tr_mal + val_mal:
    shutil.copy(file_dirs[shuffled_mal_files[index]]+'/'+ shuffled_mal_files[index],new_mal_dirs[1]+'/'+shuffled_mal_files[index])
    index += 1
while index < tr_mal + val_mal + test_mal:
    shutil.copy(file_dirs[shuffled_mal_files[index]]+'/'+ shuffled_mal_files[index],new_mal_dirs[2]+'/'+shuffled_mal_files[index])
    index += 1


import os
import pickle
import numpy as np

dirs = [shuffled_dir + '/' + 'train/Benign_Dumps',shuffled_dir + '/'+'train/Malware_Dumps']
object_count_dict = {}


for dir in dirs:
    files = os.listdir(dir)
    for file in files:
        with open(dir+"/"+file, 'rb') as f:
            if file not in object_count_dict:
                object_count_dict[file] = {}
            for line in f.readlines():
                try:
                    var = line.split()
                    obj = var[5]
                    if obj not in object_count_dict[file].keys():
                        object_count_dict[file][obj] = 0
                    object_count_dict[file][obj] += 1
                except IndexError as e:
                    continue

with open('object_count_dict_new.pkl','wb') as f:
    pickle.dump(object_count_dict,f)
    
    
import os
import pickle
import json
import numpy as np

seq_dict = {}
rev_lookup = {}
def create():
    data = {}
    with open('1.txt','r') as f:
        data = f.read()

    js = json.loads(data)
    i = 0

    for key in js.keys():
        if key not in seq_dict:
            i+=1
            seq_dict[key] = i
            rev_lookup[i] = key

    f = open('seq_dict.pkl','wb')
    pickle.dump(seq_dict,f)
    f.close()
    f = open('rev_lookup.pkl','wb')
    pickle.dump(rev_lookup,f)
    f.close()
    
def create_from_object_dict():
    i = 0
    for file,objects in object_count_dict.items():
        for obj in objects:
            key = obj.decode("utf-8")
            if key not in seq_dict:
                i+=1
                seq_dict[key] = i
                rev_lookup[i] = key
    f = open('seq_dict.pkl','wb')
    pickle.dump(seq_dict,f)
    f.close()
    f = open('rev_lookup.pkl','wb')
    pickle.dump(rev_lookup,f)
    f.close()
    
create_from_object_dict()

import os
dirs = [shuffled_dir + '/'+'train/Benign_Dumps',shuffled_dir + '/'+'train/Malware_Dumps']
file_dirs = []

for dir in dirs:
    files = os.listdir(dir)
    file_dirs.append({f:True for f in files})

In [4]:
benign_dict = {}
mal_dict = {}
for file in object_count_dict:
    is_benign = file in file_dirs[0]
    is_mal = file in file_dirs[1]
    for tok in object_count_dict[file]:
        token = tok
        if is_benign:
            benign_dict[token] = True
        elif is_mal:
            mal_dict[token] = True

In [5]:
both = {}
for token in benign_dict:
    if token in mal_dict:
        both[token] = True
        
for token in both:
    del benign_dict[token]
    del mal_dict[token]

In [6]:
def get_token_sequence(filepath):
    f = open(filepath,'r')
    lines = f.readlines()
    f.close()
    token_sequence = []
    for line in lines:
        parts = line.split('-')
        if len(parts) >= 2:
            tokens = (parts[1]).split(' ')
            if len(tokens) >=2:
                token = tokens[1]
            if token in seq_dict and token.encode("utf-8") not in both:
#                 print(token)
                token_sequence.append(seq_dict[token])
    return token_sequence

In [7]:
def vectorize_sequence_bitwise(tokenized_sample,dimensions):
    encoded_sample = np.zeros((len(tokenized_sample),dimensions))
#     print(tokenized_sample)
    for i,token in enumerate(tokenized_sample):
        bit_array = [int(c) for c in (str(bin(token))[2:][::-1])]
        for j,val in enumerate(bit_array):
            encoded_sample[i,j] = val
    return encoded_sample
def vectorize_sequence(tokenized_sample,dimensions):
    encoded_sample = np.zeros((len(tokenized_sample),dimensions))
    for i,token in enumerate(tokenized_sample):
        encoded_sample[i,token] = 1
    return encoded_sample

In [8]:
def create_dataset(benign_dir,mal_dir):

    benign_entries = [benign_dir + '/' + file for file in os.listdir(benign_dir)]
    mal_entries = [mal_dir + '/' + file for file in os.listdir(mal_dir)]
    X = []
    Y = []
    DIM = 15
    files = []

    for file_path in benign_entries:
        if not file_path.endswith('.out'):
            continue
        X.append(vectorize_sequence_bitwise(get_token_sequence(file_path),DIM))
        Y.append(0)
        files.append(file_path)

    for file_path in mal_entries:
        if not file_path.endswith('.out'):
            continue
        X.append(vectorize_sequence_bitwise(get_token_sequence(file_path),DIM))
        Y.append(1)
        files.append(file_path)

    def pad_sequences(data,max_len):
        for i,sample in enumerate(data):
            if len(sample) > max_len:
                data[i] = sample[0:max_len]
            else:
                data[i] = np.pad(data[i],((0,max_len-len(sample)),(0,0)),'constant', constant_values=((0, 0),(0,0)))
        return data

    X = pad_sequences(X,10000)

    X = np.array(X,dtype='float')
    Y = np.array(Y,dtype='float')

    return X,Y,files

In [9]:
X_train,Y_train,train_files = create_dataset(shuffled_dir + '/'+'train/Benign_Dumps',shuffled_dir + '/'+'train/Malware_Dumps')
X_test,Y_test,test_files = create_dataset(shuffled_dir + '/'+'test/Benign_Dumps',shuffled_dir + '/'+'test/Malware_Dumps')
X_validation,Y_validation,validation_files = create_dataset(shuffled_dir + '/'+'validation/Benign_Dumps',shuffled_dir + '/'+'validation/Malware_Dumps')

In [10]:
X_train = np.reshape(X_train,((X_train.shape[0],X_train.shape[1]*X_train.shape[2])))
X_test = np.reshape(X_test,((X_test.shape[0],X_test.shape[1]*X_test.shape[2])))
X_validation = np.reshape(X_validation,((X_validation.shape[0],X_validation.shape[1]*X_validation.shape[2])))

In [11]:
print(X_train.shape,X_validation.shape,X_test.shape)

(998, 150000) (200, 150000) (205, 150000)


In [12]:
from sklearn.metrics import precision_score,recall_score,f1_score

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
knn.fit(X_train, Y_train)

# print(knn.predict(x_test))
# print(knn.predict(x_train))
print("train shape: " + str(X_train.shape))
print("score on validation: " + str(knn.score(X_validation, Y_validation)))
y_val_predict = knn.predict(X_validation)
print("score on test: " + str(knn.score(X_test, Y_test)))
y_test_predict = knn.predict(X_test)
print("score on train: "+ str(knn.score(X_train, Y_train)))
y_train_predict = knn.predict(X_train)


print("Validation: ")
print("Precision : " + str(precision_score(Y_validation,y_val_predict)))
print("Recall : " + str(recall_score(Y_validation,y_val_predict)))
print("F1: " + str(f1_score(Y_validation,y_val_predict)))

print("Test: ")
print("Precision : " + str(precision_score(Y_test,y_test_predict)))
print("Recall : " + str(recall_score(Y_test,y_test_predict)))
print("F1: " + str(f1_score(Y_test,y_test_predict)))

print("Train: ")
print("Precision : " + str(precision_score(Y_train,y_train_predict)))
print("Recall : " + str(recall_score(Y_train,y_train_predict)))
print("F1: " + str(f1_score(Y_train,y_train_predict)))

train shape: (998, 150000)
score on validation: 0.74
score on test: 0.775609756097561
score on train: 0.9318637274549099
Validation: 
Precision : 0.6791044776119403
Recall : 0.91
F1: 0.7777777777777778
Test: 
Precision : 0.7092198581560284
Recall : 0.9523809523809523
F1: 0.8130081300813009
Train: 
Precision : 0.9104477611940298
Recall : 0.9870550161812298
F1: 0.9472049689440994


In [14]:
from sklearn.svm import LinearSVC
svm=LinearSVC(C=0.0001)
svm.fit(X_train, Y_train)
# print(svm.predict(x_val))
# print(svm.predict(x_train))
print("score on validation: " + str(svm.score(X_validation, Y_validation)))
y_val_predict = svm.predict(X_validation)
print("score on test: " + str(svm.score(X_test, Y_test)))
y_test_predict = svm.predict(X_test)
print("score on train: "+ str(svm.score(X_train, Y_train)))
y_train_predict = svm.predict(X_train)

print("Validation: ")
print("Precision : " + str(precision_score(Y_validation,y_val_predict)))
print("Recall : " + str(recall_score(Y_validation,y_val_predict)))
print("F1: " + str(f1_score(Y_validation,y_val_predict)))

print("Test: ")
print("Precision : " + str(precision_score(Y_test,y_test_predict)))
print("Recall : " + str(recall_score(Y_test,y_test_predict)))
print("F1: " + str(f1_score(Y_test,y_test_predict)))

print("Train: ")
print("Precision : " + str(precision_score(Y_train,y_train_predict)))
print("Recall : " + str(recall_score(Y_train,y_train_predict)))
print("F1: " + str(f1_score(Y_train,y_train_predict)))

score on validation: 0.615
score on test: 0.6292682926829268
score on train: 0.8276553106212425
Validation: 
Precision : 0.5664739884393064
Recall : 0.98
F1: 0.717948717948718
Test: 
Precision : 0.5819209039548022
Recall : 0.9809523809523809
F1: 0.7304964539007092
Train: 
Precision : 0.7822784810126582
Recall : 1.0
F1: 0.8778409090909091


In [15]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)
# print(clf.predict(x_test))
# print(clf.predict(x_train))
print("score on validation: " + str(clf.score(X_validation, Y_validation)))
y_val_predict = clf.predict(X_validation)
print("score on test: " + str(clf.score(X_test, Y_test)))
y_test_predict = clf.predict(X_test)
print("score on train: "+ str(clf.score(X_train, Y_train)))
y_train_predict = clf.predict(X_train)

print("Validation: ")
print("Precision : " + str(precision_score(Y_validation,y_val_predict)))
print("Recall : " + str(recall_score(Y_validation,y_val_predict)))
print("F1: " + str(f1_score(Y_validation,y_val_predict)))

print("Test: ")
print("Precision : " + str(precision_score(Y_test,y_test_predict)))
print("Recall : " + str(recall_score(Y_test,y_test_predict)))
print("F1: " + str(f1_score(Y_test,y_test_predict)))

print("Train: ")
print("Precision : " + str(precision_score(Y_train,y_train_predict)))
print("Recall : " + str(recall_score(Y_train,y_train_predict)))
print("F1: " + str(f1_score(Y_train,y_train_predict)))

score on validation: 0.765
score on test: 0.8
score on train: 0.9889779559118237
Validation: 
Precision : 0.7154471544715447
Recall : 0.88
F1: 0.789237668161435
Test: 
Precision : 0.75
Recall : 0.9142857142857143
F1: 0.8240343347639486
Train: 
Precision : 0.9825119236883942
Recall : 1.0
F1: 0.9911788291900561


In [16]:
from sklearn.ensemble import RandomForestClassifier
# n_estimators = number of decision trees
rf = RandomForestClassifier(n_estimators=30, max_depth=9)
rf.fit(X_train, Y_train)
print("score on validation: " + str(rf.score(X_validation, Y_validation)))
y_val_predict = rf.predict(X_validation)
print("score on test: " + str(rf.score(X_test, Y_test)))
y_test_predict = rf.predict(X_test)
print("score on train: "+ str(rf.score(X_train, Y_train)))
y_train_predict = rf.predict(X_train)

print("Validation: ")
print("Precision : " + str(precision_score(Y_validation,y_val_predict)))
print("Recall : " + str(recall_score(Y_validation,y_val_predict)))
print("F1: " + str(f1_score(Y_validation,y_val_predict)))

print("Test: ")
print("Precision : " + str(precision_score(Y_test,y_test_predict)))
print("Recall : " + str(recall_score(Y_test,y_test_predict)))
print("F1: " + str(f1_score(Y_test,y_test_predict)))

print("Train: ")
print("Precision : " + str(precision_score(Y_train,y_train_predict)))
print("Recall : " + str(recall_score(Y_train,y_train_predict)))
print("F1: " + str(f1_score(Y_train,y_train_predict)))

score on validation: 0.62
score on test: 0.624390243902439
score on train: 0.8306613226452906
Validation: 
Precision : 0.5697674418604651
Recall : 0.98
F1: 0.7205882352941176
Test: 
Precision : 0.5786516853932584
Recall : 0.9809523809523809
F1: 0.7279151943462897
Train: 
Precision : 0.7852604828462516
Recall : 1.0
F1: 0.8797153024911033
